# Jane Street Puzzle 2025 April: Sum One, Somewhere

https://www.janestreet.com/puzzles/sum-one-somewhere-index/

For a fixed p, independently label the nodes of an infinite complete binary tree 0 with probability p, and 1 otherwise. For what p is there exactly a 1/2 probability that there exists an infinite path down the tree that sums to at most 1 (that is, all nodes visited, with the possible exception of one, will be labeled 0). Find this value of p accurate to 10 decimal places.

Note that this problem is essentially a simple percolation theory program, which we can solve and explore in a variey of ways.

https://en.wikipedia.org/wiki/Percolation_theory

## Analytical Solution

The problem can easily be solved by writing down the recursive relation and working through the algebra.

### Setup

In [ ]:
import sympy

In [ ]:
p = sympy.Symbol('p')
r0 = sympy.Symbol('r_0')
r1 = sympy.Symbol('r_1')

where
- $p$ is the probability of a node being $0$
- $r_1$ is the probability that there exists an infinite path down the tree that sums to at most 1.
- $r_0$ is the probability that there exists an infinite path down the tree that sums to at most 0.

In [ ]:
r1_eq = sympy.Eq(r1, (1 - p) * (1 - (1-r0)**2) + p * (1 - (1-r1)**2))
r1_eq

In [ ]:
r0_eq = sympy.Eq(r0, p * (1 - (1-r0)**2))
r0_eq

### Solving

Perform the substitution of the given value of $r_1$ to get our set of equations.

In [ ]:
subs_eqs = [eq.subs({r1: sympy.Rational(1, 2)}) for eq in [r1_eq, r0_eq]]
subs_eqs

Solve the system of equations.

In [ ]:
p_sols = sympy.solve(subs_eqs, [p,r0], dict=True)
print(f"Found {len(p_sols)} solutions")
p_sols

Filter to only have real solutions.

In [ ]:
p_real_sols = [sol for sol in p_sols if all([v.is_real for v in sol.values()])]
print(f"Found {len(p_real_sols)} real solutions")
p_real_sols

### Analysis

Here are the numerical values of the solutions

In [ ]:
[{k:sympy.N(v) for k,v in sol.items()} for sol in p_real_sols]

Now, which solution is correct, or are both valid solutions? Well, for one we have $r_0 = 0$, which would mean it is impossible to have a sub-tree of all 0s: this corresponds to a path with a 1 infinitely far down it. We can ignore this as a degenerate case and just take the other solution.

In [ ]:
sympy.simplify(p_real_sols[1][p])

To finish off, lets get the result to 10 decimal places as requested.

In [ ]:
print(f"Final solution: p = {sympy.N(p_real_sols[1][p]):.11f}")

## Monte Carlo Simulation (Finite Depth Recursion)

We can try simulating the process of constructing a tree with random nodes and checking for an infinite path. Practically, we will need to check down to a maximum depth. One nice thing we can do is prune all infeasible branches of our search as soon as we reach them.

In [ ]:
import random

In [ ]:
def tree_sim(p, sum_rem, depth_rem):
    if depth_rem == -1:
        return True
    sum_rem -= random.random() > p
    return (sum_rem >= 0) and (tree_sim(p, sum_rem, depth_rem-1) or tree_sim(p, sum_rem, depth_rem-1))

Let's try plugging in the solution from before with a good number of iterations and depth.

In [ ]:
p_value = 0.530603575430005
depth = 100
iters = 100000

In [ ]:
samples = [tree_sim(p_value, 1, depth) for _ in range(iters)]
mean = sum(samples) / iters
mean

Let's also try the alternate solution.

In [ ]:
p_value = 2/3
depth = 100
iters = 100000

In [ ]:
samples = [tree_sim(p_value, 1, depth) for _ in range(iters)]
mean = sum(samples) / iters
mean

It doesn't appear to work, though it could be because of the finite recursion.

Note also this approach take too long to converge to act as a feasible method for computing $p$ in the first place, espcially not at the precision required. If it were though, we could use techniques like stochastic optimization.

## Dynamic Programming (Finite Depth Recursion)

We can get better resolution by working with probabilities directly with dynamic programming. Note we should use memoization/cacheing to same compute time.

In [ ]:
from functools import cache

In [ ]:
@cache
def tree_dyn(p, sum_rem, depth_rem):
    if sum_rem < 0:
        return 0
    if depth_rem == -1:
        return 1
    return p * (1 - (1-tree_dyn(p, sum_rem, depth_rem-1))**2) + (1 - p) * (1 - (1-tree_dyn(p, sum_rem-1, depth_rem-1))**2)

Now we can use pretty large depths in almost no time to get far more accurate results. This further confirms our answer.

In [ ]:
p_value = 0.530603575430005

In [ ]:
tree_dyn(p_value, 1, 1000)

In [ ]:
tree_dyn(2/3, 1, 1000)

### Solving with Dynamic Programming

We could also have solved the original problem using a root-finding algorithm, using our finite-depth dynamic programming implementation for quick computation of the function.

In [ ]:
from scipy import optimize

In [ ]:
max_depth = 1200

In [ ]:
root = optimize.root_scalar(lambda x: 1/2 - tree_dyn(x, 1, max_depth), bracket=[0, 1], xtol=10**-(10+1))
root

In [ ]:
print(f"Solution: p = {root.root:.11f}")

## Visualizations

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

### From Dynamic Programming

In [ ]:
x = np.linspace(0,1,1001)
y = np.vectorize(lambda x: tree_dyn(x, 1, 1200))(x)

In [ ]:
plt.plot(x,y)

plt.xlabel("$p$")
plt.ylabel("$r_1$")

plt.xlim(0,1)
plt.ylim(0,1)

plt.xticks(np.linspace(0,1,11))
plt.yticks(np.linspace(0,1,11))
plt.grid(True)

plt.show()

### From Analytical

Since we have the analytical solution, we could graph exactly from that:

In [ ]:
r1_sols = sympy.solve([r0_eq, r1_eq], [r0, r1], dict=True)
r1_sols

Since we have multiple solutions in terms of $p$, let's graph all of them.

In [ ]:
for i,sol in enumerate(r1_sols):
    f = sympy.lambdify(p, sympy.expand(sol[r1]), 'numpy')
    x = np.linspace(0,1,10001)
    y = f(x)
    if not isinstance(y, np.ndarray):
        y = y * np.ones_like(x)
    plt.plot(x,y, label=f"Solution {i}")

plt.xlabel("$p$")
plt.ylabel("$r_1$")

plt.xlim(0,1)
plt.ylim(0,1)

plt.xticks(np.linspace(0,1,11))
plt.yticks(np.linspace(0,1,11))
plt.grid(True)

plt.legend()

plt.show()

## Final Analysis: Domain of p

### Evaluating r1 at different p

As a finale, since we see the value of $r_1$ increases monotonically from 0 to 1, and appears to be over values of $p$ from $\frac{1}{2}$ to 1, let's verify this.

In [ ]:
print("p = 0")
for i,sol in enumerate(r1_sols):
    print(f"Solution {i}: r1 = {sol[r1].subs({p: 0})}")
print("p = 1/2")
for i,sol in enumerate(r1_sols):
    print(f"Solution {i}: r1 = {sol[r1].subs({p: sympy.Rational(1,2)})}")
print("p = 1")
for i,sol in enumerate(r1_sols):
    print(f"Solution {i}: r1 = {sol[r1].subs({p: 1})}")

So it seems the function does not give finite or real solutions for values of $p \lt \frac{1}{2}$. Let's also look at the inverse perspective.

### Evaluation p at different r1

In [ ]:
p_full_sols = sympy.solve([r0_eq, r1_eq], [p, r0], dict=True)
p_full_sols

In [ ]:
print("r1 = 0")
for i,sol in enumerate(p_full_sols):
    print(f"Solution {i}: p = {sol[p].subs({r1: 0})}")
print("r1 = 1")
for i,sol in enumerate(p_full_sols):
    print(f"Solution {i}: p = {sol[p].subs({r1: 1})}")

Let's see if we take the single sided limit if it helps.

In [ ]:
print("r1 = 0")
for i,sol in enumerate(p_full_sols):
    print(f"Solution {i}: p = {sympy.limit(sol[p], r1, 0, '+')}")
print("r1 = 1")
for i,sol in enumerate(p_full_sols):
    print(f"Solution {i}: p = {sympy.limit(sol[p], r1, 1, '-')}")

Seems not really as we still hit complex numbers.

## Conclusion

This was a fun and fairly easy problem. A great way to explore recusion, dynamic programming, Monte Carlo simulation, root-finding, and Sympy's capabilities, as well as learn about percolation theory. 

And indeed, submitting 0.53060357543 got my name on the Correct Submissions board. Thank you Jane Street for the great puzzle!